In [1]:
from peft import PeftConfig, PeftModel
from transformers import LlamaForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import os


In [6]:
config = PeftConfig.from_pretrained("GGmorello/FLAMES-20k")
model = LlamaForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    cache_dir=os.environ.get('TMPDIR'),
)
ft_model_20 = PeftModel.from_pretrained(model, "GGmorello/FLAMES-20k")

llama_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

ft_model_20 = ft_model_20.to('cuda')


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
config = PeftConfig.from_pretrained("/mimer/NOBACKUP/groups/naiss2024-23-121/morello/training")
model = LlamaForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    cache_dir=os.environ.get('TMPDIR'),
)
ft_model_100 = PeftModel.from_pretrained(model, "/mimer/NOBACKUP/groups/naiss2024-23-121/morello/training")

llama_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

ft_model_100 = ft_model_100.to('cuda')


`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

from transformers import AutoModelForCausalLM
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.tokens.instruct.request import FIMRequest
from mistral_inference.generate import generate

model_id = "mistralai/Codestral-22B-v0.1"

model = AutoModelForCausalLM.from_pretrained(model_id, cache_dir=os.environ.get('TMPDIR'),)
tokenizer = MistralTokenizer.v3()


prefix = """def add("""
suffix = """    return sum"""

request = FIMRequest(prompt=prefix, suffix=suffix)
print(request)

tokens = tokenizer.encode_fim(request)
t = torch.IntTensor(tokens.tokens)
print(tokens)

outputs = model.generate(t,max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



In [2]:
pt_model = LlamaForCausalLM.from_pretrained(
    'meta-llama/CodeLlama-7b-hf', 
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
    ),
    cache_dir=os.environ.get('TMPDIR'))
llama_tokenizer = AutoTokenizer.from_pretrained('meta-llama/CodeLlama-7b-hf')


config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

test_dataset = load_dataset('GGMorello/FLAMES', 'distilled_test', split='train', cache_dir=os.environ.get('TMPDIR'), num_proc=64).shuffle(seed=42)


Resolving data files:   0%|          | 0/155 [00:00<?, ?it/s]

Setting num_proc from 64 back to 1 for the train split to disable multiprocessing as it only contains one shard.


Generating train split: 0 examples [00:00, ? examples/s]

In [9]:
PROMPT = test_dataset[2]['input']
test_dataset[2]


{'comment': '"LazyBonesSpaceTrip: You can mint a maximum of 20 Lazy Bones"',
 'input': '// SPDX-License-Identifier: MIT\npragma solidity >=0.4.22 <0.9.0;\nimport "@openzeppelin/contracts/token/ERC721/extensions/ERC721Enumerable.sol";\nimport "@openzeppelin/contracts/token/ERC721/ERC721.sol";\nimport "@openzeppelin/contracts/security/Pausable.sol";\nimport "@openzeppelin/contracts/access/AccessControl.sol";\nimport "@openzeppelin/contracts/access/Ownable.sol";\nimport "@openzeppelin/contracts/utils/math/SafeMath.sol";\nimport "@openzeppelin/contracts/utils/Strings.sol";\ncontract LazyBonesSpaceTrip is ERC721Enumerable, Ownable, AccessControl {\n    using SafeMath for uint256;\n    uint256 public constant PreMintPrice = 0.04 ether;\n    uint256 public constant PublicMintPrice = 0.06 ether;\n    uint256 public constant TOTAL_NUMBER_OF_LAZY_BONES = 10000;\n    uint[TOTAL_NUMBER_OF_LAZY_BONES] internal indices;\n    uint internal nonce = 0;\n    uint256 public giveaway_reserved = 100;\n    

In [26]:

input_ids = llama_tokenizer(PROMPT, return_tensors="pt")["input_ids"].to('cuda')
generated_ids = ft_model.generate(input_ids, max_new_tokens=128)

filling = llama_tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
print(filling)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


payable(t6).send(_each*5)


In [27]:

input_ids = llama_tokenizer(PROMPT, return_tensors="pt")["input_ids"].to('cuda')
generated_ids = pt_model.generate(input_ids, max_new_tokens=128)

filling = llama_tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
print(filling)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


payable(t6).send(_each * 5));
  }
  function _mint(address _receiver, uint256 _mintAmount) internal {
  }
  function _mintForAddress(address _receiver, uint256 _mintAmount) internal {
  }
  function _setTokenURI(uint256 _tokenId, string memory _uri) internal {
  }
  function _setTokenURIPrefix(string memory _uriPrefix) internal {
  }
  function _setTokenURISuffix(string


In [ ]:
import torch
from tqdm import tqdm
from datasets import Dataset

results_100 = []
for data in tqdm(test_dataset.select(range(5000))):
    tok = llama_tokenizer(data['input'], return_tensors='pt').to('cuda')
    
    generated_ids = ft_model_100.generate(**tok, max_new_tokens=256, pad_token_id=llama_tokenizer.eos_token_id)
    ft_filling = llama_tokenizer.batch_decode(generated_ids[:, tok['input_ids'].shape[1]:], skip_special_tokens = True)[0]
    results_100.append(ft_filling)


 80%|███████▉  | 3983/5000 [1:07:01<15:24,  1.10it/s]  

In [ ]:
import torch
from tqdm import tqdm
from datasets import Dataset

results_20 = []
for data in tqdm(test_dataset.select(range(5000))):
    tok = llama_tokenizer(data['input'], return_tensors='pt').to('cuda')
    
    generated_ids = ft_model_20.generate(**tok, max_new_tokens=256, pad_token_id=llama_tokenizer.eos_token_id)
    ft_filling = llama_tokenizer.batch_decode(generated_ids[:, tok['input_ids'].shape[1]:], skip_special_tokens = True)[0]
    results_20.append(ft_filling)


  2%|▏         | 84/5000 [01:26<1:13:08,  1.12it/s]

In [ ]:
import torch
from tqdm import tqdm
from datasets import Dataset

results_pretrained = []
for data in tqdm(test_dataset.select(range(5000))):
    tok = llama_tokenizer(data['input'], return_tensors='pt').to('cuda')
    
    generated_ids = pt_model.generate(**tok, max_new_tokens=128, pad_token_id=llama_tokenizer.eos_token_id)
    pt_filling = llama_tokenizer.batch_decode(generated_ids[:, tok['input_ids'].shape[1]:], skip_special_tokens = True)[0]
    results_pretrained.append(pt_filling)


 60%|██████    | 3012/5000 [3:45:06<2:21:05,  4.26s/it]

In [ ]:
results_100 +=  [''] * (len(test_dataset) - len(results_100))

test_dataset = test_dataset.add_column('results', results_100)


In [ ]:
results += [''] * (len(test_dataset) - len(results))

test_dataset = test_dataset.add_column('results', results)


In [ ]:
results_pretrained += [''] * (len(test_dataset) - len(results_pretrained))

test_dataset = test_dataset.add_column('results CodeLLama', results_pretrained)

In [ ]:
test_dataset.push_to_hub('GGmorello/FLAMES_results', data_dir='data/Codellama_pretrained_5k')